## Dependencies

In [1]:
!pip install --quiet /kaggle/input/kerasapplications
!pip install --quiet /kaggle/input/efficientnet-git

In [2]:
import warnings, glob
from tensorflow.keras import Sequential, Model
import efficientnet.tfkeras as efn
from cassava_scripts import *


seed = 0
seed_everything(seed)
warnings.filterwarnings('ignore')

### Hardware configuration

In [3]:
# TPU or GPU detection
# Detect hardware, return appropriate distribution strategy
strategy, tpu = set_up_strategy()

AUTO = tf.data.experimental.AUTOTUNE
REPLICAS = strategy.num_replicas_in_sync
print(f'REPLICAS: {REPLICAS}')

REPLICAS: 1


# Model parameters

In [4]:
BATCH_SIZE = 8 * REPLICAS
HEIGHT = 512
WIDTH = 512
CHANNELS = 3
N_CLASSES = 5
TTA_STEPS = 0 # Do TTA if > 0 

# Augmentation

In [5]:
def data_augment(image, label):
    p_spatial = tf.random.uniform([], 0, 1.0, dtype=tf.float32)
    p_rotate = tf.random.uniform([], 0, 1.0, dtype=tf.float32)
#     p_pixel_1 = tf.random.uniform([], 0, 1.0, dtype=tf.float32)
#     p_pixel_2 = tf.random.uniform([], 0, 1.0, dtype=tf.float32)
#     p_pixel_3 = tf.random.uniform([], 0, 1.0, dtype=tf.float32)
    p_crop = tf.random.uniform([], 0, 1.0, dtype=tf.float32)
            
    # Flips
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    if p_spatial > .75:
        image = tf.image.transpose(image)
        
    # Rotates
    if p_rotate > .75:
        image = tf.image.rot90(image, k=3) # rotate 270º
    elif p_rotate > .5:
        image = tf.image.rot90(image, k=2) # rotate 180º
    elif p_rotate > .25:
        image = tf.image.rot90(image, k=1) # rotate 90º
        
#     # Pixel-level transforms
#     if p_pixel_1 >= .4:
#         image = tf.image.random_saturation(image, lower=.7, upper=1.3)
#     if p_pixel_2 >= .4:
#         image = tf.image.random_contrast(image, lower=.8, upper=1.2)
#     if p_pixel_3 >= .4:
#         image = tf.image.random_brightness(image, max_delta=.1)
        
    # Crops
    if p_crop > .7:
        if p_crop > .9:
            image = tf.image.central_crop(image, central_fraction=.7)
        elif p_crop > .8:
            image = tf.image.central_crop(image, central_fraction=.8)
        else:
            image = tf.image.central_crop(image, central_fraction=.9)
    elif p_crop > .4:
        crop_size = tf.random.uniform([], int(HEIGHT*.8), HEIGHT, dtype=tf.int32)
        image = tf.image.random_crop(image, size=[crop_size, crop_size, CHANNELS])
        
#     # Crops
#     if p_crop > .6:
#         if p_crop > .9:
#             image = tf.image.central_crop(image, central_fraction=.5)
#         elif p_crop > .8:
#             image = tf.image.central_crop(image, central_fraction=.6)
#         elif p_crop > .7:
#             image = tf.image.central_crop(image, central_fraction=.7)
#         else:
#             image = tf.image.central_crop(image, central_fraction=.8)
#     elif p_crop > .3:
#         crop_size = tf.random.uniform([], int(HEIGHT*.6), HEIGHT, dtype=tf.int32)
#         image = tf.image.random_crop(image, size=[crop_size, crop_size, CHANNELS])
            

    return image, label

## Auxiliary functions

In [6]:
# Datasets utility functions
def resize_image(image, label):
    ch_1 = (tf.cast(image[:, :, 0], tf.float32)  - 0.485)/ 0.229
    ch_2 = (tf.cast(image[:, :, 1], tf.float32)  - 0.456)/ 0.224
    ch_3 = (tf.cast(image[:, :, 2], tf.float32)  - 0.406)/ 0.225
    image = tf.stack([ch_1, ch_2, ch_3], axis = 2)
    
    image = tf.image.resize(image, [HEIGHT, WIDTH])
    image = tf.reshape(image, [HEIGHT, WIDTH, CHANNELS])
    return image, label

def process_path(file_path):
    name = get_name(file_path)
    img = tf.io.read_file(file_path)
    img = decode_image(img)
    img, _ = scale_image(img, None)
#     img = center_crop(img, HEIGHT, WIDTH)
    return img, name

def get_dataset(files_path, shuffled=False, tta=False, extension='jpg'):
    dataset = tf.data.Dataset.list_files(f'{files_path}*{extension}', shuffle=shuffled)
    dataset = dataset.map(process_path, num_parallel_calls=AUTO)
    if tta:
        dataset = dataset.map(data_augment, num_parallel_calls=AUTO)
    dataset = dataset.map(resize_image, num_parallel_calls=AUTO)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTO)
    return dataset

# Load data

In [7]:
database_base_path = '/kaggle/input/cassava-leaf-disease-classification/'
submission = pd.read_csv(f'{database_base_path}sample_submission.csv')
display(submission.head())

TEST_FILENAMES = tf.io.gfile.glob(f'{database_base_path}test_tfrecords/ld_test*.tfrec')
NUM_TEST_IMAGES = count_data_items(TEST_FILENAMES)
print(f'GCS: test: {NUM_TEST_IMAGES}')

,image_id,label
0,2216849948.jpg,4


GCS: test: 1


In [8]:
model_path_list = glob.glob('/kaggle/input/80-cassava-leaf-bit-resnet50x1-2-aux-adam-512/*.h5')
model_path_list.sort()

print('Models to predict:')
print(*model_path_list, sep='\n')

Models to predict:
/kaggle/input/80-cassava-leaf-bit-resnet50x1-2-aux-adam-512/model_0.h5


# Model

In [9]:
import tensorflow_hub as hub
from kaggle_datasets import KaggleDatasets

MODEL_PATH = '/kaggle/input/big-transfer-bit-weights/'

def model_fn(input_shape, N_CLASSES):
    inputs = L.Input(shape=input_shape, name='input_image')
    base_model = hub.KerasLayer(f'{MODEL_PATH}/bit_m-r50x1_1/')

    x = base_model(inputs)
    x = L.Dropout(.25)(x)
    output = L.Dense(N_CLASSES, activation='softmax', name='output')(x)
    output_healthy = L.Dense(1, activation='sigmoid', name='output_healthy')(x)
    output_cmd = L.Dense(1, activation='sigmoid', name='output_cmd')(x)
    
    model = Model(inputs=inputs, outputs=[output, output_healthy, output_cmd])

    return model

with strategy.scope():
    model = model_fn((None, None, CHANNELS), N_CLASSES)
    
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_image (InputLayer)        [(None, None, None,  0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        (None, 2048)         23500352    input_image[0][0]                
__________________________________________________________________________________________________
dropout (Dropout)               (None, 2048)         0           keras_layer[0][0]                
__________________________________________________________________________________________________
output (Dense)                  (None, 5)            10245       dropout[0][0]                    
______________________________________________________________________________________________

# Test set predictions

In [10]:
files_path = f'{database_base_path}test_images/'
test_size = len(os.listdir(files_path))
test_preds = np.zeros((test_size, N_CLASSES))


for model_path in model_path_list:
    print(model_path)
    K.clear_session()
    model.load_weights(model_path)

    if TTA_STEPS > 0:
        test_ds = get_dataset(files_path, tta=True).repeat()
        ct_steps = TTA_STEPS * ((test_size/BATCH_SIZE) + 1)
        preds = model.predict(test_ds, steps=ct_steps, verbose=1)[0][:(test_size * TTA_STEPS)]
        preds = np.mean(preds.reshape(test_size, TTA_STEPS, N_CLASSES, order='F'), axis=1)
        test_preds += preds / len(model_path_list)
    else:
        test_ds = get_dataset(files_path, tta=False)
        x_test = test_ds.map(lambda image, image_name: image)
        test_preds += model.predict(x_test)[0] / len(model_path_list)
    
test_preds = np.argmax(test_preds, axis=-1)
test_names_ds = get_dataset(files_path)
image_names = [img_name.numpy().decode('utf-8') for img, img_name in iter(test_names_ds.unbatch())]

/kaggle/input/80-cassava-leaf-bit-resnet50x1-2-aux-adam-512/model_0.h5


In [11]:
submission = pd.DataFrame({'image_id': image_names, 'label': test_preds})
submission.to_csv('submission.csv', index=False)
display(submission.head())

,image_id,label
0,2216849948.jpg,4
